# Analyzing App Store and Google Play data

This project will focus on analyzing and cleaning data from the Apple Store and Googe Plays store with an emphasis on using concepts such as basic python, lists and loops, conditional statements, dictionaries and frequency tables, functions in order to assist a theortical company design apps that will bring in revenue.
    The theortical company in question only builds apps that are free, and their revenue comes from in-app adds. Therefore we will be analyzing data from the most popular mobile devices: android and apple products. As such we will be using data from the App Store and Google Play.

[Link for the Apple Store dataset](https://www.kaggle.com/datasets/ramamet4/app-store-apple-data-set-10k-apps) (~7,200 rows and 16 columns).

 [Link for the Google Store dataset](https://www.kaggle.com/datasets/lava18/google-play-store-apps)
(~10,841 rows and 13 columns).

## Importing Data

In [1]:
# Imports the reader function from the csv module

from csv import reader

# Create a list of lists for both datasets 

open_apple = open(r'C:\Users\david\OneDrive\Desktop\AppleStore.csv', encoding="utf8")
read_apple = reader(open_apple)
apple_data = list(read_apple)
apple_header = apple_data[0]
apple_data = apple_data[1:]

open_google = open(r'C:\Users\david\OneDrive\Desktop\googleplaystoremaster.csv', encoding="utf8")
read_google = reader(open_google)
google_data = list(read_google)
google_header = google_data[0]
google_data = google_data[1:]

In [2]:
# Create a function that prints a "slice" or section of data, as well total number of rows

def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
            print(row)
            print('\n')
            
    if rows_and_columns:
            print('Number of rows:', len(dataset))
            print('Number of columns:', len(dataset[0]))
            
# Using the function about to print rows 0 through 3 with the total number of rows and columns

explore_data(google_data, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


Number of rows: 10841
Number of columns: 13


---

# Cleaning Data

There was a row in the Google Play Stores data that had a missing data point so we removed it with ```del google_data[10473]```. There are now 10,840 rows instead of 10,841 rows. Furthermore, in the cell below I created a for loop to count the number of duplicate app titles there are in the Google Play dataset. There are 1,181 duplicate app names as shown below.


In [3]:
duplicate_apps = []
unique_apps = []

del google_data[10472] # Deleted row with bad data

for app in google_data:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)

        
print('Number of duplicate apps:', len(duplicate_apps))
print('\n')
print('Examples of duplicate apps:', duplicate_apps[:10])

Number of duplicate apps: 1181


Examples of duplicate apps: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack']


-------

In order to clean our data I have created a dictionary where the duplicate datas name column and number of reviews column will be the key:value pair and out of the key:value pairs that have the highest number of reviews that pair will be placed into reviews_max. This operation will be done with every duplicate value using a for loop, as shown below.

In [4]:
reviews_max = {} # Dictionary where app names and number of reviews will go

for app in google_data:
    name = app[0]
    n_reviews = float(app[3])
    
    if name in reviews_max and reviews_max[name] < n_reviews: 
        reviews_max[name] = n_reviews
        
    elif name not in reviews_max:
        reviews_max[name] = n_reviews
        
print('Expected length:', len(google_data) - 1181)
print('Actual length:', len(reviews_max))

Expected length: 9659
Actual length: 9659


As shown above, I filtered the duplicate data using a dictionary. Our calculated number of duplicate rows was 1,181. Therefore, we would expect a clean list of data to have a length of 9,659 (original length of the google_data list after the removal of row[10472] is 10,840. So 10,840 - 1,181 = 9,659).
***
Below I created two lists, ``google_clean`` where the cleaned `google_data` will be stored and `google_app_names` for app names. The google_app_names list is a secondary condition where we filter out the results that would have gone into the `google_clean` list if the results where exactly the same. This is because our for loop above will store multiple duplicate columns if the reviews are exactly the same.

In [5]:
google_clean = []
google_app_names = []

for app in google_data:
    name = app[0]
    n_reviews = float(app[3])
    if (reviews_max[name] == n_reviews) and (name not in google_app_names):
        google_clean.append(app)
        google_app_names.append(name)

   Now that the duplicate data has been removed we need to attended do that data that is non-english. In order to accomplish this I've created a function below that counts the ASCII value of each letter in a string when inputed into said function. For further information, all english letters, as well as numbers and special characters have a ASCII number correlating to them that is under 127.
   
Example: a = 65 and ! = 33
   
   Furthermore, some app names contain symbols, dashes and emojis, yet they are usually still english based. In order to clean through app names which could pass as english-based yet have excessive amounts of emojis or special character we will limit the number of non-ASCII values by three per string as shown below.

In [6]:
def is_eng(string): # This function loops over ever letter or number and checks it's ASCII value
    non_ascii = 0
    
    for character in string:
        if ord(character) > 127:
            non_ascii += 1
            
    if non_ascii > 3:
        return False
    else: 
        return True

In [7]:
google_english = [] # In this cell we input both datasets into the function to clean it of all non-english ASCII values
apple_english = []

for row in google_clean:
    name = row[0]
    if is_eng(name):
        google_english.append(row)
        
for row in apple_data:
    name = row[1]
    if is_eng(name):
        apple_english.append(row)

Our next step is to isolate the applications on both datasets that are free, as we are only analyzing free, english-based apps in this project.

In [8]:
google_free = []
apple_free = []

for row in google_english: # For loop iterates through price in google dataset, if price is free it assigns it to a new dataset
    price = row[7]
    if price == '0':
        google_free.append(row)

for row in apple_english: # For loop iterates through price in apple dataset, if price is free it assigns it to a new dataset
    price = row[4]
    if price == '0.0':
        apple_free.append(row)
        
print('Number of free apps on Google Play: ', len(google_free))
print('Number of free apps on Apple Store: ', len(apple_free))

Number of free apps on Google Play:  8864
Number of free apps on Apple Store:  3222


---

## Analysis of App Store data

Now that we have our cleaned data for both datasets we can begin to analyze. To start I've created a function that makes frequency tables with percentages and another function to display those percentages in descending order.

In [9]:
def freq_table(dataset, index): # Frequency table for with percentages
    table = {}
    total = 0
    
    for row in dataset:
        total += 1
        value = row[index]
        if value in table:
            table[value] += 1
        else:
            table[value] = 1
    
    table_percentages = {}
    for key in table:
        percentage = (table[key] / total) * 100
        table_percentages[key] = percentage 
    
    return table_percentages


def display_table(dataset, index): # Sorts through frequency table and sorts by highest to lowest percentage
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)
        
    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

In [10]:
print('Categories in the Apple Store by percentage: ')
print('\n')
display_table(apple_free, -5)

Categories in the Apple Store by percentage: 


Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


As shown above the category that is most present in our cleaned data for the App Store is the gaming category, with a staggering approximate percentage of 58.2%. The second most common category being entertainment, with an approximate percentage of 7.9%.

To extrapolate further on the above data; it seems as if the most prominent type of applications made are those design for fun or enjoyment, rather than practical purpose. For instance, the top 5 most common categories (games, entertainment, photo & video, education and social networking) account for approximately 78% of the number of apps in our dataset (Note: this analysis does not reflect all apps on the App Store, only english-based ones). While apps that have more utility or practical usage account for the remaining percentage of apps (around 22%).
    However, the fact that the most numerous kind of apps are enjoyment-based doesn't mean they have the most number of users.

In [11]:
print('Categorys in the Google Store by percentage: ')
print('\n')
display_table(google_free, 1)

Categorys in the Google Store by percentage: 


FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661


Above is a visualization of catogries most present in Google Play, each category has a corresponding percentage for how many apps exist with these categories. As we can immediately see the Google Play store varies from the App Store in the fact that the percentages shown above are spread out amongst every genre -- even with the increased number of categories. However, we can still see some sort of  trend in fun-focused apps, even with this increased range of catgories to apps. We can see this in the family and games categories (as the family categories in Google Play is essentially just games focused for children).

In [12]:
print('\n')
print('Genres in the Google Store by percentage: ')
print('\n')
display_table(google_free, -4)



Genres in the Google Store by percentage: 


Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.936

Above is a text visualization of Genres for Google Play. The correlation between categories and genres for this data set is not crystal clear. However, we can note that there is a much large quantity of rows in the genres column than the categories coloumns. As we are focusing on the bigger picture for this analysis we will only be working with the categories column from now on.
    To recap, we've seen that the App Store has a prominent number of fun-based apps when compare to Google Play, offers a more balanced landscape between utility apps and fun-based apps.
  
 ---
 
 Our next step is to determine popularity of different categories. We will do this by counting the number of installs, unfotunately the App Store dataset lacks this column so we will be count the number of user ratings instead of installs as a next best proxy. 
 
 Below we calculate the average number of user ratings per app category in the Appe Store.

In [13]:
apple_categories = freq_table(apple_free, -5)

for category in apple_categories:
    total = 0
    len_category = 0 
    
    for app in apple_free:
        category_app = app[-5]
        if category_app == category:
            n_ratings = float(app[5])
            total += n_ratings
            len_category += 1
            
    avg_n_ratings = total / len_category # Divide the total number of apps by the number of categories
    print(category, ':', avg_n_ratings)

Social Networking : 71548.34905660378
Photo & Video : 28441.54375
Games : 22788.6696905016
Music : 57326.530303030304
Reference : 74942.11111111111
Health & Fitness : 23298.015384615384
Weather : 52279.892857142855
Utilities : 18684.456790123455
Travel : 28243.8
Shopping : 26919.690476190477
News : 21248.023255813954
Navigation : 86090.33333333333
Lifestyle : 16485.764705882353
Entertainment : 14029.830708661417
Food & Drink : 33333.92307692308
Sports : 23008.898550724636
Book : 39758.5
Finance : 31467.944444444445
Education : 7003.983050847458
Productivity : 21028.410714285714
Business : 7491.117647058823
Catalogs : 4004.0
Medical : 612.0


As show above with `Navigation : 86090.33333333333`, navigation apps have the highest number of user reviews, but this figure is heavily influenced by Waze and Google Maps, which have close to half a million user reviews together:

In [14]:
for app in apple_free:
    if app[-5] == 'Navigation':
        print(app[1], ':', app[5]) # print name and number of ratings

Waze - GPS Navigation, Maps & Real-time Traffic : 345046
Google Maps - Navigation & Transit : 154911
Geocaching® : 12811
CoPilot GPS – Car Navigation & Offline Maps : 3582
ImmobilienScout24: Real Estate Search in Germany : 187
Railway Route Search : 5


The same also applies for social networking apps, where a few giants like Facebook, Pintrest, Skype and more. This can also be seen with music apps, where Pandorda, Spotify and Shazam contain the large number of reviews compared to all the other music apps. Additionally, we can see this with genre `Reference` where the app bible and dictionary are reponsible for a large quantity of reviews as shown below.

As this analysis is supposed to help me theortical company it is important to note that the correlation found with some of these genres are because of tech giants that have so many reviews and install simply because they're a large company, and not neccesarily because the app they made alone is worth that many reviews and installs. However I will leave this data.

In [15]:
for app in apple_free:
    if app[-5] == 'Reference':
        print(app[1], ':', app[5])

Bible : 985920
Dictionary.com Dictionary & Thesaurus : 200047
Dictionary.com Dictionary & Thesaurus for iPad : 54175
Google Translate : 26786
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran : 18418
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition : 17588
Merriam-Webster Dictionary : 16849
Night Sky : 12122
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) : 8535
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools : 4693
GUNS MODS for Minecraft PC Edition - Mods Tools : 1497
Guides for Pokémon GO - Pokemon GO News and Cheats : 826
WWDC : 762
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free : 718
VPN Express : 14
Real Bike Traffic Rider Virtual Reality Glasses : 8
教えて!goo : 0
Jishokun-Japanese English Dictionary & Translator : 0


## Analysis of Google Play data

The Google Store data has a column specifically for number of installs, unlike the App Store data. However, the data is only measured in values like 100,000+ or 10,000+. Meaning that an app labeled with 200,000+ installs could actually have 200,000, 300,000, 4000,000 or more installs. For the purpose of our analysis we do not need very precise data -- just a ballpark. So we will leave this as is.

---

Below I've created a for loop that will convert the `installs` column into just numbers so that we can process it later

In [16]:
categories_google = freq_table(google_free, 1)

for category in categories_google:
    total = 0
    len_category = 0
    for app in google_free:
        category_app = app[1]
        if category_app == category:            
            n_installs = app[5]
            n_installs = n_installs.replace(',', '') # Replaces the "," in 100,000+ with a space. Therefore 100,000+ = 100000+
            n_installs = n_installs.replace('+', '') # Replaces the "+" in 100,000+ with a space. Therefore 100,000+ = 100,000
            total += float(n_installs)
            len_category += 1
    avg_n_installs = total / len_category
    print(category, ':', avg_n_installs)

ART_AND_DESIGN : 1986335.0877192982
AUTO_AND_VEHICLES : 647317.8170731707
BEAUTY : 513151.88679245283
BOOKS_AND_REFERENCE : 8767811.894736841
BUSINESS : 1712290.1474201474
COMICS : 817657.2727272727
COMMUNICATION : 38456119.167247385
DATING : 854028.8303030303
EDUCATION : 1833495.145631068
ENTERTAINMENT : 11640705.88235294
EVENTS : 253542.22222222222
FINANCE : 1387692.475609756
FOOD_AND_DRINK : 1924897.7363636363
HEALTH_AND_FITNESS : 4188821.9853479853
HOUSE_AND_HOME : 1331540.5616438356
LIBRARIES_AND_DEMO : 638503.734939759
LIFESTYLE : 1437816.2687861272
GAME : 15588015.603248259
FAMILY : 3695641.8198090694
MEDICAL : 120550.61980830671
SOCIAL : 23253652.127118643
SHOPPING : 7036877.311557789
PHOTOGRAPHY : 17840110.40229885
SPORTS : 3638640.1428571427
TRAVEL_AND_LOCAL : 13984077.710144928
TOOLS : 10801391.298666667
PERSONALIZATION : 5201482.6122448975
PRODUCTIVITY : 16787331.344927534
PARENTING : 542603.6206896552
WEATHER : 5074486.197183099
VIDEO_PLAYERS : 24727872.452830188
NEWS_AND_

As show by the above cell, communication apps have the highest number of installs at 38456119.167247385.

In [17]:
for app in google_free:
    if app[1] == 'COMMUNICATION' and (app[5] == '1,000,000,000+'
                                      or app[5] == '500,000,000+'
                                      or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

WhatsApp Messenger : 1,000,000,000+
imo beta free calls and text : 100,000,000+
Android Messages : 100,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
Messenger – Text and Video Chat for Free : 1,000,000,000+
imo free video calls and chat : 500,000,000+
Skype - free IM & video calls : 1,000,000,000+
Who : 100,000,000+
GO SMS Pro - Messenger, Free Themes, Emoji : 100,000,000+
LINE: Free Calls & Messages : 500,000,000+
Google Chrome: Fast & Secure : 1,000,000,000+
Firefox Browser fast & private : 100,000,000+
UC Browser - Fast Download Private & Secure : 500,000,000+
Gmail : 1,000,000,000+
Hangouts : 1,000,000,000+
Messenger Lite: Free Calls & Messages : 100,000,000+
Kik : 100,000,000+
KakaoTalk: Free Calls & Text : 100,000,000+
Opera Mini - fast web browser : 100,000,000+
Opera Browser: Fast and Secure : 100,000,000+
Telegram : 100,000,000+
Truecaller: Caller ID, SMS spam blocking & Dialer : 100,000,000+
UC Browser Mini -Tiny Fast Private & Secure : 100,000,000+
Viber Mess

In [18]:
print(google_header)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


As show above, a few apps account for an extremely large number of the installs like Facebook Messenger, Skype, and Gmail to name a few. If we removed the communication apps that have over 100 million installs the average would be reduced by ten times as shown below.

In [19]:
under_100_m = [] # This loops calculates the average for the the communication category if we removed the few giant apps

for app in google_free:
    n_installs = app[5]
    n_installs = n_installs.replace(',', '')
    n_installs = n_installs.replace('+', '')
    if (app[1] == 'COMMUNICATION') and (float(n_installs) < 100000000):
        under_100_m.append(float(n_installs))
        
sum(under_100_m) / len(under_100_m)

3603485.3884615386

Since the game category is so satured we wont be attempting to analyze that. The `books_and_reference` category has 8,767,811 however so we will inspect this next. Lets print out the apps and their number of installs under the `books_and_reference` category below.

In [20]:
for app in google_free: # This loop prints every app and it's number of installs as a dictionary
    if app[1] == 'BOOKS_AND_REFERENCE':
        print(app[0], ':', app[5])

E-Book Read - Read Book for free : 50,000+
Download free book with green book : 100,000+
Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Free Panda Radio Music : 100,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
English Grammar Complete Handbook : 500,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
Google Play Books : 1,000,000,000+
AlReader -any text book reader : 5,000,000+
Offline English Dictionary : 100,000+
Offline: English to Tagalog Dictionary : 500,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
Recipes of Prophetic Medicine for free : 500,000+
ReadEra – free ebook reader : 1,000,000+
Anonymous caller detection : 10,000+
Ebook Reader : 5,000,000+
Litnet - E-books : 100,000+
Read books online : 5,000,000+
English to Urdu Dictionary : 500,000+
eBoox: book reader fb2 epub zip : 1,000,000+
English Persian Dictionary : 500,000+
Flybook : 500,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
E

Immediately after creating a loop to look at the number of installs under the `books_and_reference` category I am going to filter the last for apps that have an extremely large number of installs. Lets do that below

In [21]:
for app in google_free: # This loop prints apps that have a high number of installs in the books_and_reference category
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000,000+'
                                            or app[5] == '500,000,000+'
                                            or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

Google Play Books : 1,000,000,000+
Bible : 100,000,000+
Amazon Kindle : 100,000,000+
Wattpad 📖 Free Books : 100,000,000+
Audiobooks from Audible : 100,000,000+


As shown above, it seems that there is only one giant competitor and just a few small ones in this specific category. There is potential for building and publishing an app here.

---

Lastly, lets look at the apps that are in the middle in terms of popularity to get an idea what my theortical company might profit off of the most when building an app. Let's do this in the cell below now.

In [22]:
for app in google_free: # This loop prints apps that have a moderate number of installs in the books_and_reference category
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000+'
                                            or app[5] == '5,000,000+'
                                            or app[5] == '10,000,000+'
                                            or app[5] == '50,000,000+'):
        print(app[0], ':', app[5])

Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
AlReader -any text book reader : 5,000,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
ReadEra – free ebook reader : 1,000,000+
Ebook Reader : 5,000,000+
Read books online : 5,000,000+
eBoox: book reader fb2 epub zip : 1,000,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
Moon+ Reader : 10,000,000+
English-Myanmar Dictionary : 1,000,000+
Golden Dictionary (EN-AR) : 1,000,000+
All Language Translator Free : 1,000,000+
Aldiko Book Reader : 10,000,000+
Dictionary - WordWeb : 5,000,000+
50000 Free eBooks & Free AudioBooks : 5,000,000+
Al-Quran (Free) : 10,000,000+
Al Quran Indonesia : 10,000,000+
Al'Quran Bahasa Indonesia : 10,000,000+
Al Quran Al karim : 1,000,000+
Al Quran : EAlim - Translations & MP3 Offline : 5,000,000+
Koran Read &MP3 30 Juz Offline : 1,000,000+
H

Looking through the dictionary above I can see that there seems to be a focus on dictionaries, electronic books, and audiobooks. Moreover, throughout the data analyzed there seems to be a popularity among famous books. From the Bible to the Quran. Therefore it might be worth building an app around a certain popular book. Even then though, there would still be considerable competition so the design, implementation and features would have to top of the line.

# Conclusion 

In this project, we acquired, cleaned and analyzed the data for application stores on both major mobile phone builders (android and apple), with the goal of finding out what app would become popular to maximize profits. Unfortunately, it seems many of the categories in both markets have a small number of apps that already do very well what most other apps offer.
    However, I have determined that building an app around a decently popular and well-known book with top of the line features would be an optimal choice.

In [23]:
# Read in the data
opened_file = open('Super_Bowl.csv')
read_file = opened_file.read()
# Transform read_file into a list of lists
super_bowl_split = read_file.split('\n')
super_bowl =  []
for row in super_bowl_split:
    super_bowl.append(row.split(','))

FileNotFoundError: [Errno 2] No such file or directory: 'Super_Bowl.csv'